In [1]:
!ls data/finance_sales

2003_bronx.xls        2007_statenisland.xls 2012_queens.xls
2003_brooklyn.xls     2008_bronx.xls        2012_statenisland.xls
2003_manhattan.xls    2008_brooklyn.xls     2013_bronx.xls
2003_queens.xls       2008_manhattan.xls    2013_brooklyn.xls
2003_statenisland.xls 2008_queens.xls       2013_manhattan.xls
2004_bronx.xls        2008_statenisland.xls 2013_queens.xls
2004_brooklyn.xls     2009_bronx.xls        2013_statenisland.xls
2004_manhattan.xls    2009_brooklyn.xls     2014_bronx.xls
2004_queens.xls       2009_manhattan.xls    2014_brooklyn.xls
2004_statenisland.xls 2009_queens.xls       2014_manhattan.xls
2005_bronx.xls        2009_statenisland.xls 2014_queens.xls
2005_brooklyn.xls     2010_bronx.xls        2014_statenisland.xls
2005_manhattan.xls    2010_brooklyn.xls     2015_bronx.xls
2005_queens.xls       2010_manhattan.xls    2015_brooklyn.xls
2005_statenisland.xls 2010_queens.xls       2015_manhattan.xls
2006_bronx.xls        2010_statenisland.xls 2015_queens.xls
2006_brook

In [2]:
import platform
print(platform.python_version())

3.5.2


In [3]:
import pandas as pd
import numpy as np

In [4]:
boros = ['manhattan', 'bronx', 'brooklyn', 'queens', 'statenisland']
years = range(2003, 2017)
boro_codes = dict(zip(boros, range(1,6)))

In [5]:
def read_in_boro_year_data(boro, year, data_dir = "data/finance_sales"):
    """
    Fetches data file for a specified boro and year, and returns the data as a
    Pandas dataframe. Checks integrity of boro/year arguments.

    Args:
        string boro: name of boro for desired data
        int year: year of desired data
    Returns:
        Pandas DataFrame
    """
    # Acceptable inputs
    boros = ['manhattan', 'bronx', 'brooklyn', 'queens', 'statenisland']
    years = range(2003, 2017)

    # Format input arguments appropriately
    try:
        year = int(year)
    except TypeError:
        print("inappropriate year for data")
    if year < 100:
        year = year + 2000
    assert(year in years), "inappropriate year for data"
    if boro == "si":
        boro = "statenisland"
    assert(boro in boros), "inappropriate boro for data"

    # Reads in Excel file skipping appropriate number of junk rows at the
    # beginning of file, keeping the header row as a header
    filename = "{data_dir}/{year}_{boro}.xls".format(data_dir = data_dir,
        year = year, boro = boro)
    skip_rows = 4 if year > 2010 else 3
    data = pd.read_excel(filename, skiprows = skip_rows)
    # Remove newline characters from column headers
    data.columns = [col.strip().lower() for col in data.columns]
    return data

In [6]:
brooklyn_15 = read_in_boro_year_data('brooklyn', '15')
print(brooklyn_15.columns)
print(brooklyn_15.shape)

Index(['borough', 'neighborhood', 'building class category',
       'tax class at present', 'block', 'lot', 'ease-ment',
       'building class at present', 'address', 'apartment number', 'zip code',
       'residential units', 'commercial units', 'total units',
       'land square feet', 'gross square feet', 'year built',
       'tax class at time of sale', 'building class at time of sale',
       'sale price', 'sale date'],
      dtype='object')
(25362, 21)


In [26]:
def add_BBL_and_price_per_ft(data, copy = True):
    """
    Takes a raw dataframe and adds the BBL code (Borough, Block, Lot), and
    price per square foot. Uses same 10-digit BBL format as PLUTO:
    1 digit for Borough, 5 digits for Block, 4 digits for Lot.

    Args:
        Pandas DataFrame data: raw data frame to append the "bbl" and "price
            per sqft" columns to
        boolean copy: whether to make a copy or alter the dataframe in place
    Returns:
        Pandas DataFrame
    """
    # Copy the data frame to a new object if desired
    if copy:
        processed_data = data.copy()
    else:
        processed_data = data

    # Extract the borough, block, and lot, and create a 10-digit code
    # zero-padded code from these three columns in order
    bbl_columns = data[["borough", "block", "lot"]].itertuples()
    bbl_formatted = pd.Series(["%01d%05d%04d" % (row.borough, row.block, row.lot) for row in bbl_columns], 
                              dtype='int64')
    processed_data["bbl"] = bbl_formatted
    # Use the larger: gross sqft or land sqft, gross sqft may be zero sometimes
    processed_data["price per sqft"] = data["sale price"].astype('float64') / data[[
        "gross square feet", "land square feet"]].max(axis=1)
    # Replace 0s and infinity with NaN for clarity
    processed_data = processed_data.replace(
        {'price per sqft': {0: np.nan, np.inf: np.nan}})
    return processed_data

In [27]:
brooklyn_15 = add_BBL_and_price_per_ft(brooklyn_15)
print(brooklyn_15.columns)

Index(['borough', 'neighborhood', 'building class category',
       'tax class at present', 'block', 'lot', 'ease-ment',
       'building class at present', 'address', 'apartment number', 'zip code',
       'residential units', 'commercial units', 'total units',
       'land square feet', 'gross square feet', 'year built',
       'tax class at time of sale', 'building class at time of sale',
       'sale price', 'sale date', 'bbl', 'price per sqft'],
      dtype='object')


In [28]:
def read_in_pluto(boros, data_dir = "data/nyc_pluto_16v1"):
    """
    Takes a list of boroughs and extracts PLUTO data for each borough,
    appending each subset to create a single data frame for all boroughs.

    Args:
        list(string) boros: list of all the boroughs to pull pluto data for
        string data_dir: a relative path as a string to folder containing the
            PLUTO data for all boroughs
    Returns:
        Pandas DataFrame
    """
    # mapping of how boroughs are referred in PLUTO filenames
    initials = {"manhattan" : "MN", "brooklyn" : "BK", "bronx" : "BX",
        "queens" : "QN", "statenisland" : "SI"}

    # Create an empty dataframe to store data as we iterate
    pluto = pd.DataFrame()
    for borough in boros:
        filename = "{data_dir}/{boro}.csv".format(data_dir = data_dir,
            boro = initials.get(borough))
        data = pd.read_csv(filename, low_memory = False)
        data.columns = [col.strip().lower() for col in data.columns]
        # Append new rows to existing dataframe
        pluto = pluto.append(data)
    return pluto

In [29]:
def read_in_finance(boros, years, data_dir = "data/finance_sales"):
    """
    Takes a list of boroughs and years and extracts finance data for each year,
    appending each subset to create a single data frame for all years/boroughs.

    Args:
        list(string) boros: list of all the boroughs to pull finance data for
        list(int) years: list of all the years to pull finance data for
        string data_dir: a relative path as a string to folder containing the
            department of finance sales price data for all boroughs
    Returns:
        Pandas DataFrame
    """
    # Create an empty dataframe to store data as we iterate
    finance = pd.DataFrame()
    for year in years:
        for borough in boros:
            print("Pulling Finance data for {}_{}".format(year, borough))
            boro_year = read_in_boro_year_data(borough, year, data_dir)
            boro_year = add_BBL_and_price_per_ft(boro_year)
            # Append new rows to existing dataframe
            finance = finance.append(boro_year)
    return finance

In [30]:
def merge_pluto_finance(pluto, finance, boros, years,
    output_dir = "data/merged"):
    """
    Performs an outer join on PLUTO and Dept of Finance data using BBL as the
    join key, returning a single dataframe. Also writes merged output to file.

    Args:
        Pandas DataFrame pluto: contains PLUTO data and "bbl" join key
        Pandas DataFrame finance: contains finance data and "bbl" join key
        list(string) boros: list of boroughs to use in filename of merged data
        list(int) years: list of years to use in filename of merged data
        string output_dir: directory to store merged output data
     Returns:
        Pandas DataFrame
    """
    buildings = pd.merge(pluto, finance, how='right', on='bbl',
                        suffixes=['_pluto', '_finance'])
    output = "{output_dir}/{boros_joined}_{min_year}_{max_year}.csv".format(
        boros_joined = "_".join(boros), min_year = min(years),
        max_year = max(years), output_dir = output_dir)
    buildings.to_csv(output, index = False)
    return buildings

In [31]:
boros = ['bronx']
years = [2010]

In [32]:
pluto = read_in_pluto(boros)

In [33]:
finance = read_in_finance(boros, years)

Pulling Finance data for 2010_bronx


In [34]:
buildings = merge_pluto_finance(pluto, finance, boros, years)

In [35]:
print(sum(buildings.borough_pluto.isnull()))
print(buildings.shape)

639
(4755, 108)


In [36]:
finance[["borough", "block", "lot", "bbl"]].head()

,borough,block,lot,bbl
0,2,3030,70,2030300070
1,2,3035,2,2030350002
2,2,3037,42,2030370042
3,2,3039,64,2030390064
4,2,3046,34,2030460034


In [37]:
print(len(pluto.columns))
print(len(finance.columns))
print(len(buildings.columns))

86
23
108


In [38]:
bmq_2010_2013 = pd.read_csv("data/merged/brooklyn_manhattan_queens_2010_2013.csv", low_memory=False)

In [39]:
bmq_2010_2013.head()

,borough_pluto,block_pluto,lot_pluto,cd,ct2010,cb2010,schooldist,council,zipcode,firecomp,...,commercial units,total units,land square feet,gross square feet,year built,tax class at time of sale,building class at time of sale,sale price,sale date,price per sqft
0,BK,1.0,1.0,302.0,21.0,NaN,13.0,33.0,11201.0,L118,...,0,0,151930,0,0,4,G6,9200000,2013-08-19,60.554203
1,BK,19.0,1.0,302.0,21.0,2003.0,13.0,33.0,11201.0,L118,...,42,42,41400,460000,1911,4,E1,5668200,2010-11-01,12.322174
2,BK,19.0,1.0,302.0,21.0,2003.0,13.0,33.0,11201.0,L118,...,42,42,41400,460000,1911,4,E1,5668200,2010-11-01,12.322174
3,BK,19.0,1.0,302.0,21.0,2003.0,13.0,33.0,11201.0,L118,...,42,42,41400,460000,1911,4,E1,0,2010-09-30,NaN
4,BK,20.0,21.0,302.0,21.0,3008.0,13.0,33.0,11201.0,L118,...,35,35,12600,42000,1910,4,O9,0,2012-11-28,NaN


In [40]:
buildings.head()

,borough_pluto,block_pluto,lot_pluto,cd,ct2010,cb2010,schooldist,council,zipcode,firecomp,...,commercial units,total units,land square feet,gross square feet,year built,tax class at time of sale,building class at time of sale,sale price,sale date,price per sqft
0,BX,2275.0,11.0,201.0,43.0,1000.0,7.0,8.0,10455.0,E083,...,1,11,2500,9500,1915,2,C7,670791,2010-06-14,70.609579
1,BX,2278.0,29.0,201.0,19.0,1010.0,7.0,8.0,10454.0,E060,...,0,3,1833,3120,1901,1,C0,0,2010-01-21,NaN
2,BX,2278.0,75.0,201.0,19.0,1010.0,7.0,8.0,10454.0,E060,...,1,1,5000,5000,1931,4,G2,600000,2010-02-11,120.000000
3,BX,2280.0,53.0,201.0,25.0,5002.0,7.0,8.0,10454.0,E060,...,0,10,2500,9205,1931,2,C1,790000,2010-05-18,85.822922
4,BX,2282.0,16.0,201.0,25.0,3000.0,7.0,8.0,10454.0,E060,...,1,9,2500,9293,1931,2,C7,588333,2010-04-09,63.309265


In [41]:
np.mean(buildings.lot_pluto.isnull()) # bronx 2010 how many missing pluto data

0.13438485804416403

In [42]:
np.mean(bmq_2010_2013.lot_pluto.isnull()) # bk mn qn 2010-2013 how many missing pluto data

0.32880126647522273

In [50]:
bmq_2010_2013[bmq_2010_2013.lot_pluto.isnull()][["bbl", "borough_pluto", "block_pluto", "lot_pluto", 
                                                 "borough_finance", "block_finance", "lot_finance", 
                                                 "xcoord", "ycoord", "firecomp", "policeprct", "bldgclass"]].head(20)

,bbl,borough_pluto,block_pluto,lot_pluto,borough_finance,block_finance,lot_finance,xcoord,ycoord,firecomp,policeprct,bldgclass
182311,3063970006,NaN,NaN,NaN,3,6397,6,NaN,NaN,NaN,NaN,NaN
182312,3064010012,NaN,NaN,NaN,3,6401,12,NaN,NaN,NaN,NaN,NaN
182313,3063701102,NaN,NaN,NaN,3,6370,1102,NaN,NaN,NaN,NaN,NaN
182314,3063701103,NaN,NaN,NaN,3,6370,1103,NaN,NaN,NaN,NaN,NaN
182315,3063701201,NaN,NaN,NaN,3,6370,1201,NaN,NaN,NaN,NaN,NaN
182316,3063701203,NaN,NaN,NaN,3,6370,1203,NaN,NaN,NaN,NaN,NaN
182317,3063701203,NaN,NaN,NaN,3,6370,1203,NaN,NaN,NaN,NaN,NaN
182318,3063701203,NaN,NaN,NaN,3,6370,1203,NaN,NaN,NaN,NaN,NaN
182319,3064021204,NaN,NaN,NaN,3,6402,1204,NaN,NaN,NaN,NaN,NaN
182320,3064021318,NaN,NaN,NaN,3,6402,1318,NaN,NaN,NaN,NaN,NaN


In [49]:
buildings[['bbl', 'xcoord', 'ycoord']].head()

,bbl,xcoord,ycoord
0,2022750011,1007936.0,236020.0
1,2022780029,1005461.0,232837.0
2,2022780075,1005323.0,232798.0
3,2022800053,1005803.0,233125.0
4,2022820016,1005822.0,233827.0


In [51]:
buildings[["bbl", "gross square feet", "land square feet", "sale price", "price per sqft"]]

,bbl,gross square feet,land square feet,sale price,price per sqft
0,2022750011,9500,2500,670791,70.609579
1,2022780029,3120,1833,0,NaN
2,2022780075,5000,5000,600000,120.000000
3,2022800053,9205,2500,790000,85.822922
4,2022820016,9293,2500,588333,63.309265
5,2022820027,2544,1667,470000,184.748428
6,2022840117,2160,1667,0,NaN
7,2022850004,8870,2500,742500,83.709132
8,2022850005,8870,2500,0,NaN
9,2022850006,8870,2500,0,NaN


In [55]:
buildings[["commercial units", "total units", "bbl", "sale date", 
          "gross square feet", "land square feet", "sale price", "price per sqft"]].head(20)

,commercial units,total units,bbl,sale date,gross square feet,land square feet,sale price,price per sqft
0,1,11,2022750011,2010-06-14,9500,2500,670791,70.609579
1,0,3,2022780029,2010-01-21,3120,1833,0,NaN
2,1,1,2022780075,2010-02-11,5000,5000,600000,120.000000
3,0,10,2022800053,2010-05-18,9205,2500,790000,85.822922
4,1,9,2022820016,2010-04-09,9293,2500,588333,63.309265
5,0,7,2022820027,2010-07-27,2544,1667,470000,184.748428
6,0,1,2022840117,2010-03-26,2160,1667,0,NaN
7,1,11,2022850004,2010-10-21,8870,2500,742500,83.709132
8,1,1,2022850005,2010-08-09,8870,2500,0,NaN
9,1,10,2022850006,2010-08-09,8870,2500,0,NaN
